<a href="https://colab.research.google.com/github/alyson-mei/ml_stuff/blob/alpha/NMT_25s.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural machine translation RU-EN

## Setup

In [ ]:
!pip install subword_nmt

In [209]:
import os
import random
import math
import time

import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence

from nltk.tokenize import WordPunctTokenizer

from itertools import chain
from collections import Counter
from tqdm import tqdm

In [ ]:
path_do_data = '../../datasets/Machine_translation_EN_RU/data.txt'
if not os.path.exists(path_do_data):
    print("Dataset not found locally. Downloading from github.")
    !wget https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt -nc
    path_do_data = './data.txt'

Dataset not found locally. Downloading from github.
--2025-03-03 18:56:50--  https://raw.githubusercontent.com/neychev/made_nlp_course/master/datasets/Machine_translation_EN_RU/data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12905334 (12M) [application/octet-stream]
Saving to: ‘data.txt’

data.txt            100%[===================>]  12.31M  --.-KB/s    in 0.06s   

2025-03-03 18:56:50 (190 MB/s) - ‘data.txt’ saved [12905334/12905334]



## Preprocessing

### Creating a dataset

In [ ]:
with open('data.txt') as f:
    data = [line.rstrip().split(sep = '\t')[::-1] for line in f]
for i in range(5):
    print(f'{data[i][0]} | {data[i][1]}')

Отель Cordelia расположен в Тбилиси, в 3 минутах ходьбы от Свято-Троицкого собора. | Cordelia Hotel is situated in Tbilisi, a 3-minute walk away from Saint Trinity Church.
В числе удобств лоджа Tupirmarka круглосуточная стойка регистрации и снэк-бар. Гости могут воспользоваться услугой доставки еды и напитков в номер. | At Tupirmarka Lodge you will find a 24-hour front desk, room service, and a snack bar.
Апартаменты Naigao Xiaowo расположены в городе Шанхай. К услугам гостей бесплатный Wi-Fi во всех зонах. | Featuring free WiFi in all areas, Naigao Xiaowo offers accommodation in Shanghai.
В вашем распоряжении также телевизор и собственная ванная комната с душем. | Each has a TV and a private bathroom with shower.
Номер оснащен кондиционером и спутниковым телевидением. | Your room comes with air conditioning and satellite TV.


In [ ]:
train_data, test_data = train_test_split(data, test_size = 0.2, random_state = 42)

### A simple syllables tokenizer

In [ ]:
russian_vowels = ['а', 'е', 'ё', 'и', 'о', 'у', 'ы', 'э', 'ю', 'я']
english_vowels = ['a', 'e', 'i', 'o', 'u']
vowels = set(russian_vowels + english_vowels)

In [ ]:
import string

class SyllableTokenizer:
    def __init__(self, vowels, min_word_len = 4, ru_mode = True):
        self.min_word_len = min_word_len
        self.ru_mode = ru_mode
        self.wpt_tokenizer = WordPunctTokenizer()
        self.vowels = vowels
        self.english_letters = set(string.ascii_lowercase)

    def tokenize(self, line):
        tokenized = []
        splitted_line = line.lower().split()
        for word in splitted_line:
            wpt_list = self.wpt_tokenizer.tokenize(word)
            for inner_word in wpt_list:
                if inner_word.isalpha() and len(inner_word) >= self.min_word_len and not (self.ru_mode and self.containsEnglishLetter(word)):
                    tokenized += self.splitWord(inner_word)
                else:
                    tokenized += [inner_word]
            tokenized += [' ']
        tokenized.pop()
        return tokenized

    def splitWord(self, word):
        splitted = []
        l = r = 0
        while r < len(word):
            while r < len(word) and word[r] not in self.vowels:
                r += 1
            r += 1
            while r < len(word) and word[r] not in self.vowels:
                r += 1
            if r < len(word) and word[r-1] not in self.vowels:
                r -= 1
            splitted.append(word[l:r])
            l = r
        return splitted

    def containsEnglishLetter(self, word):
        return any(char in self.english_letters for char in word)

In [ ]:
tokenizer = SyllableTokenizer(vowels = vowels)
for i in range(5):
    print('|'.join(tokenizer.tokenize(data[i][0])))

о|тель| |cordelia| |рас|по|ло|жен| |в| |тби|ли|си|,| |в| |3| |ми|ну|тах| |ходь|бы| |от| |свя|то|-|тро|иц|ко|го| |со|бо|ра|.
в| |чис|ле| |у|добств| |лод|жа| |tupirmarka| |круг|ло|су|точ|на|я| |стой|ка| |ре|гист|ра|ци|и| |и| |снэк|-|бар|.| |гос|ти| |мо|гут| |вос|поль|зо|вать|ся| |ус|лу|гой| |дос|тав|ки| |еды| |и| |на|пит|ков| |в| |но|мер|.
а|пар|та|мен|ты| |naigao| |xiaowo| |рас|по|ло|же|ны| |в| |го|ро|де| |шан|хай|.| |к| |ус|лу|гам| |гос|тей| |бесп|лат|ный| |wi|-|fi| |во| |всех| |зо|нах|.
в| |ва|шем| |рас|по|ря|же|ни|и| |так|же| |те|ле|ви|зор| |и| |собст|вен|на|я| |ван|на|я| |ком|на|та| |с| |ду|шем|.
но|мер| |ос|на|щен| |кон|ди|ци|о|не|ром| |и| |спут|ни|ко|вым| |те|ле|ви|де|ни|ем|.


### Langs + auxiliary tokens

In [ ]:
SOS, SOS_IDX = '<SOS>', 0
EOS, EOS_IDX = '<EOS>', 1
UNK, UNK_IDX = '<UNK>', 2
PAD, PAD_IDX = '<PAD>', 3

class Lang:
    def __init__(self, name, tokenize):
        self.name = name
        self.word2count = {}
        self.word2index = {SOS: SOS_IDX, EOS: EOS_IDX, UNK: UNK_IDX, PAD: PAD_IDX}
        self.index2word = {SOS_IDX: SOS, EOS_IDX: EOS, UNK_IDX: UNK, PAD_IDX: PAD}
        self.n_tokens = 4
        self.n_tokens_trimmed = self.n_tokens
        self.tokenize = tokenize

    def addSentence(self, sentence):
        for token in self.tokenize(sentence):
            self.addWord(token)

    def addWord(self, token):
        if token not in self.word2index:
            self.word2count[token] = 1
            self.word2index[token] = self.n_tokens
            self.n_tokens += 1
            self.n_tokens_trimmed += 1
            self.index2word[self.n_tokens] = token
        else:
            self.word2count[token] += 1

    def trimDict(self, min_freq = 3):
        for token in self.word2count:
            if self.word2count[token] < min_freq:
                self.word2index[token] = UNK_IDX
                self.n_tokens_trimmed -= 1

    def normalizeWord2Index(self):
        idx = 4
        word2index = {SOS: SOS_IDX, EOS: EOS_IDX, UNK: UNK_IDX, PAD: PAD_IDX}
        index2word = {SOS_IDX: SOS, EOS_IDX: EOS, UNK_IDX: UNK, PAD_IDX: PAD}
        for token in self.word2count:
            if self.word2index[token] != UNK_IDX:
                word2index[token] = idx
                index2word[idx] = token
                idx += 1
            else:
                word2index[token] = UNK_IDX
        self.word2index = word2index
        self.index2word = index2word

In [ ]:
tokenize_src = SyllableTokenizer(vowels = vowels).tokenize
tokenize_tgt = lambda sentence: WordPunctTokenizer().tokenize(sentence.lower())

lang_src = Lang("src", tokenize_src)
lang_tgt = Lang("tgt", tokenize_tgt)

for src, tgt in tqdm(train_data):
     lang_src.addSentence(src)
     lang_tgt.addSentence(tgt)
lang_src.trimDict(min_freq = 2)
lang_tgt.trimDict(min_freq = 2)
for src, _ in tqdm(test_data):
     lang_src.addSentence(src)

lang_src.normalizeWord2Index()
lang_tgt.normalizeWord2Index()

100%|██████████| 10000/10000 [00:00<00:00, 13699.45it/s]


In [ ]:
print(f'Initial src vocab size: {lang_src.n_tokens}, trimmed src vocab size: {lang_src.n_tokens_trimmed}')
print(f'Initial dst vocab size: {lang_tgt.n_tokens}, trimmed dst vocab size: {lang_tgt.n_tokens_trimmed}')

Initial src vocab size: 21458, trimmed src vocab size: 9615
Initial dst vocab size: 27271, trimmed dst vocab size: 10126


### Indexing

In [ ]:
def indexingSrc(src, lang_src = lang_src):
    src = tokenize_src(src)
    return [SOS_IDX] + [lang_src.word2index[token] for token in src] + [EOS_IDX]
def indexingTgt(tgt, lang_dst = lang_tgt):
    tgt = tokenize_tgt(tgt)
    return [SOS_IDX] + [lang_dst.word2index[token] for token in tgt] + [EOS_IDX]

def translateSrc(indexed_src, lang_src = lang_src):
    return [lang_src.index2word[token] for token in indexed_src]
def translateTgt(indexed_dst, lang_dst = lang_tgt):
    return [lang_tgt.index2word[token] for token in indexed_dst]

In [ ]:
src, tgt = random.sample(train_data, 1)[0]

indexed_src = indexingSrc(src)
indexed_tgt = indexingTgt(tgt)
translated_src = translateSrc(indexed_src)
translated_tgt = translateTgt(indexed_tgt)

print(src, indexed_src, translated_src)
print(tgt, indexed_tgt, translated_tgt)

Поездка от апартаментов до порта Албуфейры занимает 5 минут, а знаменитая набережная Оура-Стрип находится в 2,3 км. [0, 103, 411, 42, 6, 32, 6, 54, 168, 58, 159, 268, 6, 41, 6, 57, 58, 6, 1087, 439, 1232, 199, 6, 261, 35, 449, 69, 6, 157, 6, 27, 350, 25, 6, 54, 6, 33, 34, 35, 58, 45, 6, 10, 224, 1395, 10, 45, 6, 4, 80, 18, 40, 2, 6, 10, 11, 12, 13, 6, 14, 6, 506, 25, 318, 6, 71, 43, 1] ['<SOS>', 'по', 'езд', 'ка', ' ', 'от', ' ', 'а', 'пар', 'та', 'мен', 'тов', ' ', 'до', ' ', 'пор', 'та', ' ', 'ал', 'бу', 'фей', 'ры', ' ', 'за', 'ни', 'ма', 'ет', ' ', '5', ' ', 'ми', 'нут', ',', ' ', 'а', ' ', 'зна', 'ме', 'ни', 'та', 'я', ' ', 'на', 'бе', 'реж', 'на', 'я', ' ', 'о', 'у', 'ра', '-', '<UNK>', ' ', 'на', 'хо', 'дит', 'ся', ' ', 'в', ' ', '2', ',', '3', ' ', 'км', '.', '<EOS>']
Albufeira Marina is a 5-minute drive and the famous Oura Strip is 2.3 km away. [0, 1247, 1078, 13, 14, 89, 16, 17, 338, 43, 20, 21, 3232, 7207, 13, 374, 24, 209, 33, 86, 24, 1] ['<SOS>', 'albufeira', 'marina', 'is

In [ ]:
def indexingData(data, test = False):
    indexed_data = []

    if not test:
        for src, tgt in tqdm(data):
            indexed_data.append([indexingSrc(src), indexingTgt(tgt)])
        return sorted(indexed_data, key = lambda x: (len(x[0]), len(x[1])))
    else:
        if len(np.shape(data)) == 2:
            for src, _ in tqdm(data):
                indexed_data.append(indexingSrc(src))
        else:
            for src in tqdm(data):
                indexed_data.append(indexingSrc(src))
        return sorted(indexed_data, key = lambda x: len(x))

In [205]:
indexed_train_data = indexingData(train_data)
indexed_test_data = indexingData(test_data, test = True)

100%|██████████| 10000/10000 [00:03<00:00, 3037.84it/s]


In [ ]:
print(indexed_train_data[0])
print(indexed_test_data[0])

[[0, 43, 1], [0, 24, 1]]
[0, 326, 327, 6, 238, 43, 1]


### Making batches

In [215]:
BATCH_SIZE = 128

def prepareEpoch(indexed_data, batch_size = BATCH_SIZE, pad_idx = PAD_IDX, N = 40000):
    indices = np.sort(np.random.choice(len(indexed_data), size = N, replace = False))
    epoch_indexed_data = []
    for i in range(N):
        epoch_indexed_data.append(indexed_data[indices[i]])

    i = 0
    batches = []
    while i <= len(epoch_indexed_data) - batch_size:
        src_list = []
        dst_list = []
        for j in range(i, i + batch_size):
            src_list.append(torch.LongTensor(epoch_indexed_data[j][0]))
            dst_list.append(torch.LongTensor(epoch_indexed_data[j][1]))
        batch_src = pad_sequence(src_list, padding_value = pad_idx)
        batch_dst = pad_sequence(dst_list, padding_value = pad_idx)
        batches.append([batch_src, batch_dst])
        i += batch_size

    return batches

In [216]:
batches = prepareEpoch(indexed_train_data)
len(batches)

312

## experimental part

In [ ]:
!pip install gensim


In [208]:
src_train[:10]

['Отель Days Inn Liverpool находится в деловом районе Ливерпуля, в 10 минутах ходьбы от знаменитого Альберт-Дока.',
 'Расстояние до ближайшего международного аэропорта имени Нетаджи Субхас Чандра Боса составляет 16 км.',
 'К услугам гостей мягкий уголок с диванами и телевизором и полностью оборудованная собственная кухня с плитой, холодильником и посудой.',
 'В распоряжении гостей отеля Genipabu Praia круглосуточная стойка регистрации, сад и терраса.',
 'В лаконично оформленных номерах имеются телевизор с кабельными каналами и тумбочки с лампами.',
 'Отель типа «постель и завтрак» apple расположен в 32 км от Центра Ирландского хрусталя в Голуэе и в 40 км от стадиона «Голуэй-Грейхаунд».',
 'Гостиница «Шах» расположена в Астане, в 5 км от монумента «Байтерек». К услугам гостей бесплатный Wi-Fi, бесплатная частная парковка, спа-центр, сауна и ресторан.',
 'Кроме того, гостям предоставляется бесплатная парковка и бесплатный WiiFi во всех областях.',
 'В вашем распоряжении собственная ванна

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

# Пример корпуса (список предложений)

# Токенизация предложений

src_train = [x[0] for x in train_data]
tokenized_sentences = [tokenize_src(sentence.lower()) for sentence in src_train]

# Создание корпуса и вычисление совместной встречаемости слов
model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)




In [ ]:
!pip install umap-learn

In [206]:
words = list(model.wv.index_to_key)  # Список всех слов
embeddings = [model.wv[word] for word in words]  # Соответствующие эмбеддинги
from sklearn.manifold import TSNE
import numpy as np

# Преобразуем эмбеддинги в массив numpy
embeddings_array = np.array(embeddings)

# Уменьшаем размерность до 2D с помощью t-SNE
from umap import UMAP

umap = UMAP(n_components=2, random_state=42)
embeddings_2d = umap.fit_transform(embeddings_array)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [207]:
import plotly.express as px
import pandas as pd

# Создаём DataFrame для визуализации
df = pd.DataFrame({
    'x': embeddings_2d[:, 0],
    'y': embeddings_2d[:, 1],
    'word': words
})

# Интерактивный scatter plot
fig = px.scatter(df, x='x', y='y', text='word', title="Визуализация Word2Vec с помощью t-SNE")
fig.update_traces(textposition='top center')
fig.show()
